# LSTM for model estimation

## Imports

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error
import seaborn as sb

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Data
Source for dataset files :
https://intofpv.com/t-hi-need-some-tips-on-my-10-setup?pid=89730#pid89730

In [2]:
#data = pd.read_csv("A4E.csv")
data = pd.read_csv("./data/FAST FLYING APRIL 17.01.csv")
#print(data.columns)

In [3]:
# Check our data quickly
%matplotlib ipympl
time = (data[' time (us)'] - 22006993)/1e6 #this is in sync with betaflight's blackbox explorer
plt.plot(time,data[' gyroADC[0]'])
plt.title('Gyro x axis measurement')
plt.ylabel('deg/s')
plt.show()

plt.figure()
plt.plot(time,data[' motor[0]'])
plt.title('Motor 0 command')
plt.show()

print("fe : ", len(data[' motor[0]'])/np.max(time))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

fe :  1997.9499367601159


Here the sampling frequency is quite high at 2 kHz. We will downsample to be able to load data representative of the system in the RAM (ie if we take 20 samples per prediction at 2 kHz, the window time span will be very short in regards with the system's dynamics.


Our goal is to model the system with its input and its output.
Here our input to the model is the previous states of the system, and previous+current 4 motor commands. Motor commands here seems to be between 207 and 2047, so we will need to scale them.
The state of the system here will be the output of the gyro.


In [10]:
X = np.empty((data_length - WINDOW_LENGTH-1, WINDOW_LENGTH, nb_features))
Y = data_output[WINDOW_LENGTH + 1:] #this is equivalent to shifting every element so that Y[t] contains gyro values of t+1; the first value will not be used

#pre-computing
delayed = []
for k in range(WINDOW_LENGTH):
    delay = k+1
    delayed.append(np.roll(data_input, delay, axis=0))

for k in range(data_length - WINDOW_LENGTH-1):
    print('Status [%d%%]\r'%(1+100*k/(data_length - WINDOW_LENGTH-1)), end="")
    for i in range(WINDOW_LENGTH):
        X[k, i] = delayed[WINDOW_LENGTH-i-1][k+WINDOW_LENGTH]

In [11]:
print(X.shape, Y.shape)

(25628, 200, 7) (25628, 3)


In [12]:
def scale(X, meanX, stdX):
    return (X - meanX) / stdX

def inverse_scale(X, meanX, stdX):
    return X * stdX + meanX

In [13]:
from sklearn.model_selection import train_test_split

#this sklearn function shuffles the data
x, X_test, y, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
X_train, X_validate, y_train, y_validate = train_test_split(x, y, test_size=0.33, random_state=0)
print("Train samples :", len(X_train))
print("Test samples :", len(X_test))
print("Validation samples :", len(X_validate))

train_size = len(X_train)

input_mean, input_std = X_train.reshape((-1, 7)).mean(axis=0), X_train.reshape((-1, 7)).std(axis=0),
output_mean, output_std = y_train.mean(axis=0), y_train.std(axis=0)
print(input_mean, output_mean)
validation_dataset = tf.data.Dataset.from_tensor_slices((scale(X_validate, input_mean, input_std), scale(y_validate, output_mean, output_std)))
train_dataset = tf.data.Dataset.from_tensor_slices((scale(X_train, input_mean, input_std), scale(y_train, output_mean, output_std)))
test_dataset = tf.data.Dataset.from_tensor_slices((scale(X_test, input_mean, input_std), scale(y_test, output_mean, output_std)))

Train samples : 15453
Test samples : 2563
Validation samples : 7612
[-14.40394616  -8.12071442  14.92381285 697.80150294 606.0505905
 719.35758526 631.98102699] [-14.21510386  -8.12845402  14.966479  ]


In [33]:
#del X
#del Y

In [154]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

BATCH_SIZE = 32
batched_validation_dataset = validation_dataset.batch(BATCH_SIZE)
batched_train_dataset = train_dataset.batch(BATCH_SIZE)
batched_test_dataset = test_dataset.batch(BATCH_SIZE)

model = Sequential()
model.add(LSTM(200, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(8, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
#model.add(Flatten(input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(Dense(512, activation="relu"))
#model.add(Dense(64, activation="relu"))
model.add(Dense(3, activation="linear"))

opt = tf.keras.optimizers.Adam(learning_rate=0.1, amsgrad=True)
model.compile(loss='mse', optimizer="adam")

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 200, 200)          166400    
_________________________________________________________________
dropout_7 (Dropout)          (None, 200, 200)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 8)                 6688      
_________________________________________________________________
dropout_8 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 3)                 27        
Total params: 173,115
Trainable params: 173,115
Non-trainable params: 0
_________________________________________________________________


In [155]:
history = model.fit(
    batched_train_dataset,
    epochs = 50, 
    batch_size = BATCH_SIZE, 
    validation_data= batched_validation_dataset
    )

Epoch 1/50
483/483 [==============================] - 10s 18ms/step - loss: 0.3132 - val_loss: 0.0736
Epoch 2/50
483/483 [==============================] - 10s 20ms/step - loss: 0.1396 - val_loss: 0.0489
Epoch 3/50
483/483 [==============================] - 9s 18ms/step - loss: 0.1179 - val_loss: 0.0362
Epoch 4/50
483/483 [==============================] - 8s 17ms/step - loss: 0.1035 - val_loss: 0.0251
Epoch 5/50
483/483 [==============================] - 8s 17ms/step - loss: 0.1021 - val_loss: 0.0291
Epoch 6/50
483/483 [==============================] - 9s 19ms/step - loss: 0.1057 - val_loss: 0.0268
Epoch 7/50
483/483 [==============================] - 9s 18ms/step - loss: 0.0960 - val_loss: 0.0208
Epoch 8/50
483/483 [==============================] - 8s 17ms/step - loss: 0.0940 - val_loss: 0.0246
Epoch 9/50
483/483 [==============================] - 8s 17ms/step - loss: 0.0910 - val_loss: 0.0194
Epoch 10/50
483/483 [==============================] - 8s 17ms/step - loss: 0.0958 - val_

In [156]:
model.evaluate(batched_test_dataset)

81/81 [==============================] - 1s 9ms/step - loss: 0.0185


0.018463555723428726

In [157]:
plt.figure()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.title('Mean Square Error per Epoch')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [158]:
print(len(X))

25628


In [159]:
nb_values = 1000
start = 12000

inp = np.empty((nb_values, WINDOW_LENGTH, 7))
inp[:] = X[start:start+nb_values]
prediction = model.predict(scale(inp, input_mean, input_std))
plt.figure()

plt.plot(scale(Y[start:start+nb_values], output_mean, output_std)[:,0])
plt.plot(prediction[:,0])

plt.title("One step prediction, x axis")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [160]:
prediction = model.predict(scale(inp, input_mean, input_std))

In [161]:
print(prediction[50])
print(scale(Y[12050], output_mean, output_std))

[ 0.26494864 -0.06860718 -0.34931275]
[ 0.30214157 -0.05461136 -0.42566582]


In [162]:
def predictMultipleSteps(history_frame, inputs, true):
    res = []
    current_frame = history_frame
    shape = history_frame.shape
    print(shape)
    for ind, u in enumerate(inputs):
        print('Status [%d%%]\r'%(1+100*ind/(len(inputs))), end="")
        
        p = model.predict(current_frame)
        #print(p, true[ind])
        current_frame_last = np.hstack((p[0], u))
        current_frame = np.vstack((current_frame[0, 1:], current_frame_last)).reshape(shape)
        res.append(p[0])
    return np.array(res)

In [163]:
START = 13000
STEPS = 200

history_frame = X[START].reshape((1, -1, nb_features))

inputs = scale(X[START+1:START+1+STEPS], input_mean, input_std)[:,-1, 3:]

prediction_multiple_steps = predictMultipleSteps(scale(history_frame, input_mean, input_std), inputs, scale(Y[START:START+STEPS], output_mean, output_std))


(1, 200, 7)


In [164]:
AXIS = 1

plt.figure()
plt.title("Prediction on the Y axis")
plt.plot(prediction_multiple_steps[:,AXIS], label="Prediction")
plt.plot(scale(Y[START:START+STEPS], output_mean, output_std)[:,AXIS],label="True value")
plt.legend()
#plt.plot(scale(data_input[12001:12101, :3], output_mean, output_std)[:,0])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …